<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Finance_MULTIPLE_LLM_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai -q
!pip install anthropic -q
!pip install colab-env -q

In [2]:
import os
from typing import Dict, Any, List, Optional
import google.generativeai as genai
from anthropic import Anthropic
import colab_env # For Google Colab

# Import necessary libraries
try:
    import google.generativeai as genai
    from anthropic import Anthropic
    from anthropic.types import MessageParam
    # Conditional import for Colab's userdata
    try:
        from google.colab import userdata
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False
        print("Running outside Google Colab. 'userdata.get' will not be available.")

except ImportError:
    print("\n[CRITICAL ERROR] Please install necessary libraries:")
    print("pip install google-generativeai anthropic")
    print("Exiting as API functionality cannot be demonstrated without them.")
    # In a real application, you might raise an exception or handle this more gracefully
    # For this example, we'll allow execution but API calls will be mocked
    pass


# --- 1. Core Components (Interfaces and Implementations with API Management) ---

class BaseModel:
    """Base class for Language Models and Reasoning Models."""
    def __init__(self, name: str, identifier: str, api_client: Any):
        self.name = name
        self.identifier = identifier # The specific model string
        self.api_client = api_client # The actual API client instance

    def generate(self, prompt: str, **kwargs) -> str:
        raise NotImplementedError("Subclasses must implement 'generate'")

class LargeLanguageModel(BaseModel):
    """Integrates with models/gemini-2.0-flash API (or similar LLM)."""
    def __init__(self, name: str, identifier: str, api_client: Optional[genai.GenerativeModel]):
        super().__init__(name, identifier, api_client)
        if self.api_client is not None and not isinstance(self.api_client, genai.GenerativeModel):
             print(f"Warning: Expected genai.GenerativeModel for {identifier}, but received {type(api_client)}. Mocking API calls.")
             self.api_client = None

    def generate(self, prompt: str, temperature: float = 0.7, max_tokens: int = 500) -> str:
        """
        Makes an actual API call or uses mock response.
        """
        print(f"[{self.name} ({self.identifier})] Attempting to generate response for prompt: '{prompt[:80]}...'")

        if self.api_client:
            try:
                generation_config = {
                    "temperature": temperature,
                    "max_output_tokens": max_tokens,
                }
                response = self.api_client.generate_content(
                    prompt,
                    generation_config=generation_config
                )
                if response and hasattr(response, 'text') and response.text:
                    return response.text
                elif response and hasattr(response, 'prompt_feedback'):
                    print(f"Prompt feedback from LLM: {response.prompt_feedback}")
                    return f"API did not generate content due to safety concerns or other issues."
                return f"API generated an empty response."
            except Exception as e:
                print(f"Error calling LLM API for {self.identifier}: {e}")
                return self._mock_generate(prompt)
        else:
            return self._mock_generate(prompt) # Fallback to mock if client not initialized

    def _mock_generate(self, prompt: str) -> str:
        """Mock response when API is not available or fails."""
        print(f"[{self.name} ({self.identifier})] (MOCK) Generating response for prompt: '{prompt[:80]}...'")
        if "financial report" in prompt.lower() and "company a" in prompt.lower():
            return f"[{self.name}] (MOCK) Analyzing financial report for Company A. Key metrics identified: Revenue, Profit Margin, Debt-to-Equity. Need to retrieve historical data."
        elif "investment strategy" in prompt.lower() and "risk tolerance" in prompt.lower():
             return f"[{self.name}] (MOCK) Considering investment strategy based on user's risk tolerance. Potential assets include stocks, bonds, and mutual funds. Need to retrieve market data."
        return f"[{self.name}] (MOCK) Generated a general financial response based on: '{prompt}'"


class LargeReasoningModel(BaseModel):
    """Integrates with claude-opus-4-20250514 API (or similar LRM)."""
    def __init__(self, name: str, identifier: str, api_client: Optional[Anthropic]):
        super().__init__(name, identifier, api_client)
        if self.api_client is not None and not isinstance(self.api_client, Anthropic):
             print(f"Warning: Expected anthropic.Anthropic for {identifier}, but received {type(api_client)}. Mocking API calls.")
             self.api_client = None


    def generate(self, prompt: str, complexity_level: str = "high") -> str:
        """
        Makes an actual API call or uses mock response.
        """
        print(f"[{self.name} ({self.identifier})] Attempting to perform reasoning for prompt: '{prompt[:80]}...' (Complexity: {complexity_level})")

        if self.api_client:
            try:
                messages: List[MessageParam] = [
                    {"role": "user", "content": prompt}
                ]
                response = self.api_client.messages.create(
                    model=self.identifier,
                    max_tokens=1000,
                    messages=messages,
                    temperature=0.1,
                )
                if response and hasattr(response, 'content') and response.content:
                    return "".join(block.text for block in response.content if hasattr(block, 'text'))
                return f"API generated an empty response."
            except Exception as e:
                print(f"Error calling LRM API for {self.identifier}: {e}")
                return self._mock_generate(prompt)
        else:
            return self._mock_generate(prompt) # Fallback to mock if client not initialized

    def _mock_generate(self, prompt: str) -> str:
        """Mock response when API is not available or fails."""
        print(f"[{self.name} ({self.identifier})] (MOCK) Performing financial reasoning for prompt: '{prompt[:80]}...'")
        if "analyze financial data" in prompt.lower() and "company a" in prompt.lower():
            return f"[{self.name}] (MOCK) LRM reasoned: Analyzed Company A's financials. Identified growth in Q3 revenue but a slight decrease in profit margin due to increased operating costs. Calculated key ratios."
        elif "recommend investment" in prompt.lower():
            return f"[{self.name}] (MOCK) LRM reasoned: Evaluated market conditions and user risk tolerance. Recommended a diversified portfolio with exposure to tech and healthcare sectors based on current trends and historical performance data."
        return f"[{self.name}] (MOCK) LRM performed complex financial reasoning based on: '{prompt}'"

# --- Mock Retrieval Tools (Adapted for Finance) ---

class Memory:
    """Manages short-term and long-term memory."""
    def __init__(self):
        self.short_term_memory: List[str] = []
        self.long_term_memory: List[str] = [] # Could be a more complex data structure

    def add_short_term(self, entry: str):
        self.short_term_memory.append(entry)
        # Keep short-term memory limited
        if len(self.short_term_memory) > 10:
            self.short_term_memory.pop(0)

    def add_long_term(self, entry: str):
        self.long_term_memory.append(entry)
        # In a real system, this might involve embeddings and indexing for financial data

    def retrieve_short_term(self, query: str = "") -> List[str]:
        # Simple retrieval; in real system, would be more sophisticated (e.g., semantic search)
        return [m for m in self.short_term_memory if query.lower() in m.lower()]

    def retrieve_long_term(self, query: str = "") -> List[str]:
        return [m for m in self.long_term_memory if query.lower() in m.lower()]

class VectorDatabase:
    """Mock Vector Database (Adapted for Finance)."""
    def query(self, embedding: List[float], top_k: int = 5) -> List[str]:
        print(f"Querying Vector DB with embedding (simulating financial data search)...")
        # Simulate results related to financial data
        return ["company_a_financial_statements_vdb", "tech_sector_performance_data_vdb"]

class SemanticDatabase:
    """Mock Semantic Database (Adapted for Finance)."""
    def query(self, natural_language_query: str) -> List[str]:
        print(f"Querying Semantic DB for: '{natural_language_query}' (simulating financial knowledge search)...")
        # Simulate results related to financial regulations or concepts
        if "regulations" in natural_language_query or "compliance" in natural_language_query:
            return ["SEC Regulation S-X requirements for financial statements.", "Guidelines for insider trading compliance."]
        elif "investment terms" in natural_language_query or "metrics" in natural_language_query:
             return ["Explanation of Price-to-Earnings (P/E) Ratio.", "Definition of EBITDA."]
        return ["financial_concept_1_sdb", "market_definition_2_sdb"]

class SearchEngine:
    """Mock Search Engine (e.g., Google, Financial News Sites - Adapted)."""
    def search(self, query: str) -> List[str]:
        print(f"Searching external sources for: '{query}' (simulating financial web search)...")
        if "stock price apple" in query.lower():
            return ["Current AAPL stock price: $175.50 (as of market close).", "News: Apple announces new product line."]
        elif "inflation rate" in query.lower():
            return ["Current US Inflation Rate: 3.4% (CPI, April 2024).", "Historical inflation data chart."]
        return [f"Financial web result for '{query}' - Article A", f"Financial web result for '{query}' - Report B"]

# --- 2. Retrieval Agents ---

class RetrievalAgent:
    """Base class for retrieval agents."""
    def __init__(self, name: str):
        self.name = name

    def retrieve(self, query: str, **kwargs) -> List[str]:
        raise NotImplementedError("Subclasses must implement 'retrieve'")

class KnowledgeBaseRetrievalAgent(RetrievalAgent):
    """Retrieves from Vector and Semantic Databases (Adapted for Finance)."""
    def __init__(self, vector_db: VectorDatabase, semantic_db: SemanticDatabase):
        super().__init__("FinancialKnowledgeAgent")
        self.vector_db = vector_db
        self.semantic_db = semantic_db

    def retrieve(self, query: str, query_type: str = "semantic") -> List[str]:
        results = []
        if query_type == "vector":
            # In a real system, 'query' would first be embedded, likely into a financial context embedding
            mock_embedding = [0.2] * 768 # Placeholder financial embedding
            results.extend(self.vector_db.query(mock_embedding))
        elif query_type == "semantic":
            results.extend(self.semantic_db.query(query))
        return results

class SearchEngineRetrievalAgent(RetrievalAgent):
    """Retrieves from external search engines (Adapted for Finance)."""
    def __init__(self, search_engines: List[SearchEngine]):
        super().__init__("FinancialSearchAgent")
        self.search_engines = search_engines

    def retrieve(self, query: str) -> List[str]:
        all_results = []
        for engine in self.search_engines:
            all_results.extend(engine.search(query))
        return all_results


# --- 3. Agentic Reasoning and Console (Adapted for Finance) ---

class AgenticFinancialReasoning:
    """Orchestrates LLM, LRM, and Retrieval Agents for Financial Tasks."""
    def __init__(self, llm_general: LargeLanguageModel,
                 lrm_financial: LargeReasoningModel, memory: Memory,
                 kb_agent: KnowledgeBaseRetrievalAgent, se_agent: SearchEngineRetrievalAgent):
        self.llm_general = llm_general
        self.lrm_financial = lrm_financial
        self.memory = memory
        self.kb_agent = kb_agent
        self.se_agent = se_agent
        # We'll use the general LLM as the primary interpreter
        self.current_llm = self.llm_general

    def process_query(self, modified_input: str) -> str:
        """
        This method embodies the core Agentic Reasoning loop for financial queries.
        """
        print(f"\n[Agentic Financial Reasoning] Processing input using {self.current_llm.identifier} and {self.lrm_financial.identifier}: '{modified_input[:80]}...'")
        self.memory.add_short_term(f"User query: {modified_input}")

        # Step 1: Initial LLM interpretation and plan using the general LLM
        llm_system_prompt_initial = "You are an AI financial analyst assistant. Analyze the user query to determine information needs and initial analysis steps for financial tasks. Prioritize retrieving relevant financial data."
        initial_llm_thought = self.current_llm.generate(f"{llm_system_prompt_initial}\nUser query: {modified_input}")
        self.memory.add_short_term(f"LLM initial thought: {initial_llm_thought}")
        print(f"LLM Initial Thought: {initial_llm_thought}")

        # Step 2: Determine retrieval needs based on LLM's thought (Financial Keywords)
        retrieval_query = ""
        # Keywords updated for finance
        financial_keywords = ["financial data", "market data", "stock price", "company report", "investment", "regulation", "economic indicator", "financial statement"]
        if any(keyword in initial_llm_thought.lower() for keyword in financial_keywords):
             retrieval_query = modified_input # Use original query or refine it
             print(f"Determined retrieval need for financial information based on: '{retrieval_query[:80]}'")

             # Perform Retrieval using Financial Agents
             kb_results = self.kb_agent.retrieve(retrieval_query, query_type="semantic") # Assume semantic for most financial queries
             se_results = self.se_agent.retrieve(retrieval_query)
             retrieved_info = "\n".join(kb_results + se_results)
             print(f"Retrieved Financial Information: {retrieved_info[:200]}...")
             self.memory.add_short_term(f"Retrieved: {retrieved_info}")
        else:
             retrieved_info = "No specific financial retrieval needed."
             print(retrieved_info)


        # Step 3: Engage LRM (Financial Reasoning Model) if complex reasoning is required
        lrm_system_prompt = "You are a specialized financial reasoning engine (claude-opus-4-20250514). Analyze the gathered financial information and the user's original intent to formulate a precise financial analysis or recommendation. Focus on accuracy and relevant financial metrics."
        # Keywords updated for financial reasoning tasks
        reasoning_keywords = ["analyze", "evaluate", "compare", "recommend", "predict", "strategy", "model"]
        if any(keyword in modified_input.lower() for keyword in reasoning_keywords) or any(keyword in initial_llm_thought.lower() for keyword in reasoning_keywords):
             reasoning_input = f"User query: {modified_input}\nInitial LLM thought: {initial_llm_thought}\nRetrieved info: {retrieved_info}"
             lrm_output = self.lrm_financial.generate(f"{lrm_system_prompt}\n{reasoning_input}")
             self.memory.add_short_term(f"LRM financial reasoning: {lrm_output}")
             print(f"LRM Financial Output: {lrm_output}")
        else:
             lrm_output = "No specific complex financial reasoning performed by claude-opus-4-20250514."
             print(lrm_output)


        # Step 4: Final LLM Synthesis (General LLM)
        final_llm_prompt = f"Based on the user's original financial query: '{modified_input}', initial analysis: '{initial_llm_thought}', retrieved financial information: '{retrieved_info}', and specialized financial reasoning by {self.lrm_financial.identifier}: '{lrm_output}', synthesize a comprehensive, accurate, and professional financial answer using {self.current_llm.name}."
        final_response = self.current_llm.generate(final_llm_prompt, max_tokens=1000)
        self.memory.add_long_term(f"Interaction: Query='{modified_input}', Response='{final_response}'")
        print(f"\n[Final Financial Response] {final_response}")
        return final_response

class AgenticFinancialConsole:
    """The top-level console orchestrating user interaction for Financial Queries."""
    def __init__(self, gemini_client: Optional[genai.GenerativeModel], claude_client: Optional[Anthropic]):
        # Initialize general LLM
        self.llm_general = LargeLanguageModel("Gemini 2.0 Flash (General)", "gemini-2.0-flash", gemini_client)
        # Initialize Financial Reasoning Model
        self.lrm_financial = LargeReasoningModel("Claude Opus 4 (Financial Reasoning)", "claude-opus-4-20250514", claude_client)

        self.memory = Memory() # Memory class remains general
        self.vector_db = VectorDatabase() # Mock DB adapted for finance
        self.semantic_db = SemanticDatabase() # Mock DB adapted for finance
        self.search_engines = [SearchEngine()] # Mock Search Engine adapted for finance
        # Use Financial-specific Retrieval Agents
        self.kb_agent = KnowledgeBaseRetrievalAgent(self.vector_db, self.semantic_db)
        self.se_agent = SearchEngineRetrievalAgent(self.search_engines)

        # Use AgenticFinancialReasoning
        self.agentic_reasoning = AgenticFinancialReasoning(
            llm_general=self.llm_general,
            lrm_financial=self.lrm_financial,
            memory=self.memory,
            kb_agent=self.kb_agent,
            se_agent=self.se_agent
        )

    def run_query(self, user_query: str) -> str:
        print(f"\n--- [Agentic Financial Console] Processing User Query: '{user_query}' ---")
        parsed_query = user_query.strip()
        print(f"Query Parsed: '{parsed_query}'")

        # System Prompt for the overall financial task
        system_prompt = "You are a helpful AI financial analyst assistant. Your goal is to provide accurate and comprehensive financial analysis and information."
        modified_input = f"{system_prompt}\nUser: {parsed_query}"

        # Agentic Reasoning for Financial Query
        compiled_output = self.agentic_reasoning.process_query(modified_input)

        print(f"--- [Agentic Financial Console] Query Complete ---")
        return compiled_output

# --- Example Usage (Adapted for Financial Query) ---
if __name__ == "__main__":
    print("Setting up API clients...")

    # --- API Setup remains the same ---
    # This part retrieves API keys and initializes clients.
    # Ensure your environment variables 'GEMINI' and 'ANTHROPIC_API_KEY' are set.

    GOOGLE_API_KEY = None
    if IN_COLAB:
        try:
            GOOGLE_API_KEY = userdata.get('GEMINI')
            print("Successfully retrieved GEMINI_API_KEY from Colab environment variable.")
        except Exception as e:
            print(f"Could not retrieve GEMINI_API_KEY from Colab userdata: {e}. Checking environment variable.")
    if not GOOGLE_API_KEY:
        GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY") # Fallback to environment variable
        if GOOGLE_API_KEY:
            print("Successfully retrieved GOOGLE_API_KEY from environment variable.")
        else:
            print("WARNING: GOOGLE_API_KEY not found. Gemini API calls will be mocked.")

    gemini_client = None
    if GOOGLE_API_KEY:
        try:
            genai.configure(api_key=GOOGLE_API_KEY)
            # Use the model identifier string as expected by the API
            gemini_client = genai.GenerativeModel('gemini-1.5-flash') # Changed to a generally available model if 'gemini-2.0-flash' isn't standard
            print("Gemini GenerativeModel ('gemini-1.5-flash') client initialized.")
        except Exception as e:
            print(f"ERROR initializing Gemini client: {e}. Gemini API calls will be mocked.")
    else:
        print("No Google API Key found, Gemini client will not be initialized.")

    CLAUDE_API_KEY = None
    if IN_COLAB:
        try:
             # Note: Colab userdata uses string names. Check the actual name used for your key.
            CLAUDE_API_KEY = userdata.get("ANTHROPIC_API_KEY") # Or whatever name you used
            print("Successfully retrieved ANTHROPIC_API_KEY from Colab userdata.")
        except Exception as e:
            print(f"Could not retrieve ANTHROPIC_API_KEY from Colab userdata: {e}. Checking environment variable.")
    if not CLAUDE_API_KEY:
        CLAUDE_API_KEY = os.environ.get("ANTHROPIC_API_KEY") # Fallback to environment variable
        if CLAUDE_API_KEY:
            print("Successfully retrieved ANTHROPIC_API_KEY from environment variable.")
        else:
            print("WARNING: ANTHROPIC_API_KEY not found. Claude API calls will be mocked.")

    claude_client = None
    # Note: Using a generally available Claude model identifier
    CLAUDE_MODEL_IDENTIFIER = "claude-3-5-sonnet-20240620" # Or "claude-3-opus-20240229" or another available model
    if CLAUDE_API_KEY:
        try:
            claude_client = Anthropic(api_key=CLAUDE_API_KEY)
            print(f"Anthropic Claude client initialized for model '{CLAUDE_MODEL_IDENTIFIER}'.")
        except Exception as e:
            print(f"ERROR initializing Anthropic client: {e}. Claude API calls will be mocked.")
    else:
        print("No Anthropic API Key found, Claude client will not be initialized.")


    # Instantiate the console with the prepared API clients
    # This line creates the main console object that orchestrates everything.
    # Pass the Claude model identifier to the console constructor
    console = AgenticFinancialConsole(gemini_client=gemini_client, claude_client=claude_client)
    print('\n')
    print("\nAgenticFinancialConsole initialized. Running your specific financial query...")
    print('\n')

    # --- Running Your Specific Financial Query ---
    your_financial_query = "Analyze the recent financial performance of Company A based on available reports and market data, and provide a brief outlook."

    print(f"\n--- Running Financial Query: {your_financial_query} ---")
    financial_output = console.run_query(your_financial_query)
    print(f"\n--- Financial Analysis Output ---")
    print(financial_output)
    print("--- End of Financial Analysis Output ---")

    # --- Displaying Memory Contents after your query ---
    print("\n--- Current Short-Term Memory ---")
    print(console.agentic_reasoning.memory.retrieve_short_term())
    print('\n')

    print("\n--- Current Long-Term Memory ---")
    print(console.agentic_reasoning.memory.retrieve_long_term())
    print('\n')

Mounted at /content/gdrive
Setting up API clients...
Could not retrieve GEMINI_API_KEY from Colab userdata: Requesting secret GEMINI timed out. Secrets can only be fetched when running from the Colab UI.. Checking environment variable.
Successfully retrieved GOOGLE_API_KEY from environment variable.
Gemini GenerativeModel ('gemini-1.5-flash') client initialized.
Could not retrieve ANTHROPIC_API_KEY from Colab userdata: Secret ANTHROPIC_API_KEY does not exist.. Checking environment variable.
Successfully retrieved ANTHROPIC_API_KEY from environment variable.
Anthropic Claude client initialized for model 'claude-3-5-sonnet-20240620'.



AgenticFinancialConsole initialized. Running your specific financial query...



--- Running Financial Query: Analyze the recent financial performance of Company A based on available reports and market data, and provide a brief outlook. ---

--- [Agentic Financial Console] Processing User Query: 'Analyze the recent financial performance of Company A based